In [1]:
import sys,os
sys.path.append(os.path.expanduser('~/smodels'))
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'
import numpy as np
from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.theory.branch import Branch
from smodels.tools.coverage import Uncovered,UncoveredGroup,GeneralElement
from smodels.theory.auxiliaryFunctions import index_bisect
%load_ext line_profiler
setLogLevel("error")

# Set the path to the database
database = Database(os.path.expanduser("~/smodels/test/database"))

### Load model

In [2]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 1*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               doCompress=True, doInvisible=True,
                               minmassgap=mingap)

In [4]:
print(len(topDict.getElements()))
print(topDict.getTotalWeight())

917
['8.00E+00 [TeV]:4.75E+00 [pb] (None, None)', '1.30E+01 [TeV]:1.40E+01 [pb] (None, None)']


### Load the experimental results to be used.

In [5]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)


 Theory Predictions and Constraints:

 ATLAS-CONF-2013-037 
r = 1.142E-01

 ATLAS-SUSY-2013-02 
r = 6.560E-05

 ATLAS-SUSY-2013-12 
r = 6.460E-01

 CMS-SUS-13-012 
r = 5.050E-03


In [7]:
# import time
# t0 = time.time()
uncovered = Uncovered(topDict, sqrts=13.*TeV)
# print(time.time()-t0)

In [8]:
# First sort coverage groups by label
groups = sorted(uncovered.groups[:], key=lambda g: g.label)
# Print uncovered cross-sections:
for group in groups:
    print("\nTotal cross-section for %s (fb): %10.3E\n" % (group.description, group.getTotalXSec()))


Total cross-section for missing topologies (fb):  1.046E+04


Total cross-section for missing topologies with displaced decays (fb):  0.000E+00


Total cross-section for missing topologies with prompt decays (fb):  1.046E+04


Total cross-section for topologies outside the grid (fb):  8.055E+01



In [20]:
missingTopos = uncovered.getGroup('missing (prompt)')
finalStates = {}
if missingTopos.generalElements:
    for iel,genEl in enumerate(missingTopos.generalElements[:]):
        elStr = str(genEl).replace(']','').replace('[','')
        elStr = elStr.replace('(',',').replace(')','')
        elStr = elStr.replace(' ','')
        elStr = ','.join(sorted([ee for ee in elStr.split(',') if ee]))
        if elStr not in finalStates:
            finalStates[elStr] = genEl.missingX
        else:
            finalStates[elStr] += genEl.missingX
else:
    print("No missing topologies found\n")

# Print some of the missing topologies:
f = open('coverage-old.txt','w')
print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))
    f.write('%s : %1.3e\n' %(fs, np.round(finalStates[fs],4)))
f.close()        

148
Element: MET,MET
	cross-section (fb): 3.8833
Element: MET,MET,W,W,jet,jet,jet,jet,jet,jet
	cross-section (fb): 130.8067
Element: MET,MET,W,Z,b,b,b,jet,t
	cross-section (fb): 4.7111
Element: MET,MET,W,Z,b,jet,jet,jet,jet,jet,t
	cross-section (fb): 34.8003
Element: MET,MET,W,Z,b,jet,jet,jet,t
	cross-section (fb): 9.8184
Element: MET,MET,W,Z,b,jet,t
	cross-section (fb): 4.8514
Element: MET,MET,W,Z,jet,jet,jet,jet
	cross-section (fb): 61.0672
Element: MET,MET,W,Z,jet,jet,l,nu
	cross-section (fb): 9.0134
Element: MET,MET,W,Z,jet,jet,nu,ta
	cross-section (fb): 4.4948
Element: MET,MET,W,b,b,b,b,b,b,b,t
	cross-section (fb): 7.8995
Element: MET,MET,W,b,b,b,b,b,b,jet,jet
	cross-section (fb): 8.5326
Element: MET,MET,W,b,b,b,b,b,jet,jet,t
	cross-section (fb): 58.3254
Element: MET,MET,W,b,b,b,b,b,t
	cross-section (fb): 22.0898
Element: MET,MET,W,b,b,b,b,jet,jet
	cross-section (fb): 15.0736
Element: MET,MET,W,b,b,b,b,jet,jet,jet
	cross-section (fb): 88.9474
Element: MET,MET,W,b,b,b,b,jet,jet,jet

In [10]:
missingDisplaced = uncovered.getGroup('missing (displaced)')
print(len(missingDisplaced.generalElements))
# Print elements with displaced decays:
if missingDisplaced.generalElements:
    print('\nElements with displaced vertices (up to 2):')
    for genEl in missingDisplaced.generalElements[:2]:
        print('Element:', genEl)
        print('\tcross-section (fb):', genEl.missingX)
else:
    print("\nNo displaced decays")


0

No displaced decays


In [11]:
missingGrid = uncovered.getGroup('outsideGrid (all)')
print(len(missingGrid.generalElements))
# Print some of the missing topologies:
if missingGrid.generalElements:
    for genEl in missingGrid.generalElements[:]:
#         if str(genEl) != '(PV > MET,MET,jet,jet,l,nu)':
#             continue
        print('Element:', genEl)
        print('\tcross-section (fb):', np.round(genEl.missingX,4))
#         for el in genEl._contributingElements:
#             print(el.tree.treeToBrackets())
#             el.drawTree()
else:
    print("No missing topologies found\n")

2
Element: [[[b,b]],[[b,b]]]  (MET,MET)
	cross-section (fb): 72.7112
Element: [[[t,t]],[[b,b]]]  (MET,MET)
	cross-section (fb): 7.8367


In [12]:
genEl = missingTopos.generalElements[9]
print(len(genEl._contributingElements))
for el in genEl._contributingElements:
    print(el,el.weight.getMaxXsec())

4
[[[b,b]],[[b,b],[q,q]]] 1.54E-01 [pb]
[[[b,b]],[[b,b],[c,c]]] 4.29E-02 [pb]
[[[b,b]],[[b,b],[q,q]]] 4.17E-02 [pb]
[[[b,b]],[[b,b],[c,c]]] 1.16E-02 [pb]


In [13]:
for el in topDict.getElements():
    if str(el) == '[[[c,c]],[[b,b],[b,b]]]':
        print(el,el.testedBy,el.coveredBy,el.weight.getMaxXsec())
        break

[[[c,c]],[[b,b],[b,b]]] set() set() 3.29E-03 [pb]


In [14]:
for mom in el.motherElements:
    print(mom,mom.elID)
    print(mom.oddParticles)
    print(mom.weight.getMaxXsec())

[[[c,c]],[[b,b],[b,b]]] 159
[[gluino, N1], [gluino, N2, N1]]
3.29E-03 [pb]
[[[c,c]],[[b,b],[b,b]]] 0
[[gluino, N1], [gluino, N2, N1]]
1.64E-03 [pb]


In [15]:
missingTopos.getMissingX(el),missingTopos.reweightFactor(el)

(3.2894097755385134, 1.0)